In [22]:
import sys
sys.path.append('../src')
# from causal_shapley import causal_shapley, predict_proba
from egtoolkit import *
%load_ext autoreload
%autoreload 2

from egtoolkit import *

import random
import warnings
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import itertools
from itertools import combinations, permutations, chain
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mutual_info_score

import shap

warnings.filterwarnings("ignore")
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.under_sampling import RandomUnderSampler

from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete.CPD import TabularCPD

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import dataset

In [66]:
categorical_cols = ['global_active_power', 'global_reactive_power', 'voltage',
                    'global_intensity', 'kitchen', 'laundry', 'climate_control', 'other',
                    'weekend', 'month_name', 'season_name', 'day_name']  # list all columns that are categorical

In [67]:
dataset = pd.read_csv('./datasets/2.0-discretized-v2-3-peak-label-encoded.csv')

In [68]:
dataset[['peak_warning']].values

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [69]:
targets = ['peak_label_pred', 'peak_warning', 'no_significant_change', 'lower_than_usual']
targets_only = ['peak_label_pred', 'no_significant_change', 'lower_than_usual']

In [70]:
dataset.columns

Index(['global_active_power', 'global_reactive_power', 'voltage',
       'global_intensity', 'kitchen', 'laundry', 'climate_control', 'other',
       'weekend', 'month_name', 'season_name', 'day_name', 'peak_label_pred',
       'peak_warning', 'no_significant_change', 'lower_than_usual'],
      dtype='object')

MI Sort

In [77]:
causal_vars = ['climate_control','global_intensity','kitchen','laundry','other','voltage','weekend', 'peak_warning']
dataset_preprocessing = dataset[causal_vars]
dataset_preprocessing

,climate_control,global_intensity,kitchen,laundry,other,voltage,weekend,peak_warning
0,High,Very High,High,Medium,Very High,Low,True,False
1,High,High,Low,Low,High,High,False,False
2,Very High,Very High,Medium,Medium,Very High,High,False,False
3,Low,Low,Medium,Low,Medium,Low,False,False
4,Low,Very High,Low,Low,Very High,Very High,True,False
...,...,...,...,...,...,...,...,...
41668,Low,Low,Low,Low,Medium,Very High,True,False
41669,Medium,Medium,High,Medium,High,Very High,False,False
41670,Low,Medium,Low,High,High,High,False,False
41671,Medium,Low,High,High,Low,Very High,True,False


In [78]:
# Initialize LabelEncoder
le = LabelEncoder()

# Apply LabelEncoder to each column
for col in dataset_preprocessing.columns:
    dataset_preprocessing[col] = le.fit_transform(dataset_preprocessing[col])

In [79]:
# Prepare an empty DataFrame to store mutual information
mi_matrix = pd.DataFrame(index=dataset_preprocessing.columns, columns=dataset_preprocessing.columns)

# Calculate mutual information for each pair of columns
for col1 in dataset_preprocessing.columns:
    for col2 in dataset_preprocessing.columns:
        mi_matrix.loc[col1, col2] = mutual_info_score(dataset_preprocessing[col1], dataset_preprocessing[col2])

print(mi_matrix.values)


[[1.3748624052229603 0.2918502082238087 0.3196853707866573
  0.06627524257671642 0.037659607081020396 0.0503604623530874
  0.0006325003465987022 0.09426270891192148]
 [0.2918502082238087 1.3733068821830432 0.04214078617042706
  0.046260120346218125 0.41624975079542736 0.10048726791888057
  0.002922910585734295 0.15097777384148847]
 [0.3196853707866573 0.04214078617042706 1.38286802432115
  0.260686249324554 0.010542826604020783 0.044340309486752896
  0.0023339804933382724 0.020583936653749024]
 [0.06627524257671642 0.046260120346218125 0.260686249324554
  1.3854443909985528 0.007645556847585777 0.030925773163837364
  0.0010848848257274668 0.005839022333192917]
 [0.037659607081020396 0.4162497507954274 0.010542826604020783
  0.007645556847585777 1.379847170104834 0.061686982120370173
  0.0022612178958185125 0.06990186563545549]
 [0.0503604623530874 0.10048726791888057 0.044340309486752896
  0.030925773163837364 0.061686982120370173 1.3791596177645482
  0.00013741577370590674 0.084805339

In [80]:
influence_scores = mi_matrix.sum() - np.diag(mi_matrix)
sorted_features = influence_scores.sort_values(ascending=False)
print("Features sorted by influence score:")
print(sorted_features)

Features sorted by influence score:
global_intensity    1.050889
climate_control     0.860726
kitchen             0.700313
other               0.605948
peak_warning        0.426687
laundry             0.418717
voltage             0.372744
weekend             0.009689
dtype: object


In [83]:
mi_matrix['peak_warning']['voltage']

0.08480533965934756

K2 Algorithm

In [62]:
import numpy as np
from math import factorial as fact

def calculate_score(data, parents, child_index, ri, qi, alpha_ijk):
    """
    Calculate the score of a particular node and its parents.
    
    :param data: The dataset containing the observations.
    :param parents: The current set of parent indices for the node.
    :param child_index: The index of the child node.
    :param ri: The number of possible values for the child variable.
    :param qi: The number of unique instantiations of the parents.
    :param alpha_ijk: The count of occurrences of child and parent instantiations.
    
    :return: The calculated score.
    """
    score = 0
    for j in range(qi):
        Nij = sum(alpha_ijk[j])
        score += np.log(fact(ri - 1) / fact(Nij + ri - 1))
        for k in range(ri):
            score += np.log(fact(alpha_ijk[j][k]))
    return score

def k2_algorithm(data, node_order, max_parents):
    """
    Implements the K2 algorithm for learning the structure of a Bayesian Network.
    
    :param data: The dataset to learn from.
    :param node_order: The order in which to consider the nodes.
    :param max_parents: The maximum number of parents a node can have.
    
    :return: The structure of the Bayesian Network (list of parents for each node).
    """
    n = data.shape[1]  # Number of nodes
    network_structure = [[] for _ in range(n)]

    for i, node in enumerate(node_order):
        # Set of possible parents for this node is all nodes preceding 'node' in node_order
        possible_parents = node_order[:i]
        best_score = float('-inf')
        node_parents = []

        # Iterate through possible parents combinations
        while len(node_parents) < max_parents and possible_parents:
            for new_parent in possible_parents:
                # Test score with the new parent added
                current_parents = node_parents + [new_parent]
                # Calculate the score with the new parent set
                # Note: You would need to implement calculate_alpha_ijk, calculate_ri, and calculate_qi based on your data
                ri = calculate_ri(data, node)
                qi = calculate_qi(data, current_parents)
                alpha_ijk = calculate_alpha_ijk(data, current_parents, node, ri, qi)
                current_score = calculate_score(data, current_parents, node, ri, qi, alpha_ijk)

                # Update best score and structure
                if current_score > best_score:
                    best_score = current_score
                    node_parents = current_parents

            # Remove the added parent from possible parents
            possible_parents = [parent for parent in possible_parents if parent not in node_parents]

        network_structure[node] = node_parents

    return network_structure


In [84]:
def calculate_ri(data, node_index):
    """ Calculate the number of unique values (ri) for the child variable. """
    return len(np.unique(data[:, node_index]))

def calculate_qi(data, parent_indices):
    """ Calculate the number of unique instantiations (qi) of the parent set. """
    if not parent_indices:
        return 1
    # Get all unique rows of parent values
    unique_parents = np.unique(data[:, parent_indices], axis=0)
    return unique_parents.shape[0]

def calculate_alpha_ijk(data, parent_indices, child_index, ri, qi):
    """ Calculate the counts (alpha_ijk) of child/parent instantiations. """
    # Initialize alpha_ijk
    alpha_ijk = np.zeros((qi, ri), dtype=int)
    
    # If there are no parents, count the occurrences of each child value
    if not parent_indices:
        for k in range(ri):
            alpha_ijk[0, k] = np.sum(data[:, child_index] == k)
    else:
        # Get unique parent instantiations
        unique_parents = np.unique(data[:, parent_indices], axis=0)
        # Count occurrences of each child value for each parent instantiation
        for j, parent_instantiation in enumerate(unique_parents):
            # Filter data for the current parent instantiation
            parent_mask = np.all(data[:, parent_indices] == parent_instantiation, axis=1)
            for k in range(ri):
                alpha_ijk[j, k] = np.sum(data[parent_mask, child_index] == k)
    
    return alpha_ijk


In [86]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from math import factorial as fact
from itertools import product

# Encode the categorical variables
le = LabelEncoder()
encoded_data = np.array([le.fit_transform(col) for col in zip(*dataset_preprocessing)]).T

# Define the node ordering (this is just an example - you should define your own order based on domain knowledge or other criteria)
node_order = list(range(encoded_data.shape[1]))

def calculate_ri(data, node_index):
    """Calculate the number of unique values for the child variable."""
    return len(np.unique(data[:, node_index]))

def calculate_qi(data, parent_indices):
    """Calculate the number of unique instantiations of the parent set."""
    if not parent_indices:
        return 1
    unique_parent_instantiations = np.unique(data[:, parent_indices], axis=0)
    return len(unique_parent_instantiations)

def calculate_alpha_ijk(data, parent_indices, child_index, ri, qi):
    """Calculate the counts of child/parent instantiations."""
    if not parent_indices:
        # Count occurrences of each value for the child node
        return np.array([np.sum(data[:, child_index] == k) for k in range(ri)])
    else:
        # Count occurrences for each parent instantiation and child value
        alpha_ijk = np.zeros((qi, ri), dtype=int)
        unique_parent_instantiations = np.unique(data[:, parent_indices], axis=0)
        for j, parent_instantiation in enumerate(unique_parent_instantiations):
            for k in range(ri):
                mask = np.all(data[:, parent_indices] == parent_instantiation, axis=1)
                alpha_ijk[j, k] = np.sum(data[mask, child_index] == k)
        return alpha_ijk

# Replace the k2_algorithm() function with the tailored version for your dataset
# ...

# Use the tailored K2 algorithm function to learn the structure
network_structure = k2_algorithm(encoded_data, node_order, max_parents=2)
